In [0]:
USER = 'dobos'
PREFIX = '/datascope/subaru'

GALAXY = 'fornax'
OBS_PATH = '/datascope/subaru/data/cmdfit/dSph'
HSC_FILE = f'/datascope/subaru/data/cmdfit/dSph/{GALAXY}_tpall3e_g24.cat'
GAIA_FILE = '/datascope/subaru/data/cmdfit/dSph/gaia.h5'
SKY_FILE = f'/datascope/subaru/data/cmdfit/dSph/sky_{GALAXY}.feather'
FLUXSTD_FILE = f'/datascope/subaru/data/cmdfit/dSph/fluxstd_{GALAXY}.feather'
# MLCLASS_FILE = '/datascope/subaru/data/targeting/dSph/umi/ursaminor_mlclass.csv'
# PMAP_FILE = '/datascope/subaru/data/cmdfit/run/umi/sim/nobin_chab_nb_250k_001/pmap.h5'

# GALAXY = 'umi'
# OBS_PATH = '/datascope/subaru/data/cmdfit/dSph'
# HSC_FILE = '/datascope/subaru/data/cmdfit/dSph/umi_tpall3e_g24.cat'
# GAIA_FILE = '/datascope/subaru/data/cmdfit/dSph/gaia.h5'
# SKY_FILE = '/datascope/subaru/data/cmdfit/dSph/sky_ursaminor.feather'
# FLUXSTD_FILE = '/datascope/subaru/data/cmdfit/dSph/fluxstd_ursaminor.feather'
# MLCLASS_FILE = '/datascope/subaru/data/targeting/dSph/umi/ursaminor_mlclass.csv'
# PMAP_FILE = '/datascope/subaru/data/cmdfit/run/umi/sim/nobin_chab_nb_250k_001/pmap.h5'

#date of observation
YEAR = 2024
MONTH = 10
DAY = 25

ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

PN_FILE = f'{PREFIX}/data/catalogs/PN/NGC6822_catalogue_for_pfs.dat'

GAIA_CROSSMATCH_RADIUS = 0.1    # in arcsec

NVISITS = 1
OUTPUT_PATH = f'/datascope/subaru/user/dobos/netflow/{GALAXY}_{NVISITS}_visit'
# OUTPUT_PATH = f'/datascope/subaru/user/dobos/netflow/{GALAXY}_{NVISITS}_visit_pn'
# OUTPUT_PATH = f'/datascope/subaru/user/dobos/netflow/fornax_{NVISITS}_visit'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
# if 'debug' not in globals():
#     import debugpy
#     debugpy.listen(('0.0.0.0', 5698))
#     debug = True

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

# Set up plotting

In [0]:
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow, Visit

In [0]:
# galaxy = GALAXIES[GALAXY]
galaxy = GALAXIES['for']
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]
pointings

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Generate pointings and visits

This should match the netflow settings

In [0]:
# Observation time is the next day from now at midnight
obs_time = datetime(YEAR, MONTH, DAY, hour=0, minute=30, second=0, microsecond=0) + pd.Timedelta(hours=10)
obs_time = Time(obs_time)
print('obs_time', obs_time)

In [0]:
pointings = []
visits = []
exp_time = 12 / NVISITS * 900

# Observation time is the next day from now at midnight
# obs_time = datetime.now() + pd.Timedelta(days=1)
# obs_time = obs_time.replace(hour=0, minute=0, second=0, microsecond=0)
obs_time = datetime(YEAR, MONTH, DAY, hour=0, minute=30, second=0, microsecond=0) + pd.Timedelta(hours=10)
obs_time = Time(obs_time)
print('obs_time', obs_time)

vidx = 0
for pidx, p in enumerate((galaxy.get_pointings(SubaruPFI))):
    # Need to convert targeting.pointing to netflow.pointing
    np = Pointing(
        p.ra, p.dec,
        posang=p.posang,
        # obs_time=Time(datetime.now()),

        # Make sure the target is visible at the time of observation!
        obs_time=obs_time,
        exp_time = exp_time,
        nvisits=NVISITS
    )
    pointings.append(np)

    for i in range(np.nvisits):
        nv = Visit(vidx, pidx, np, visit_cost=0)
        visits.append(nv)
        vidx += 1

len(pointings), len(visits)

# Load the data

In [0]:
from pfs.ga.targeting.data import Observation, Catalog
from pfs.ga.targeting.io import TextObservationReader, DataFrameSerializer
from pfs.ga.targeting.util.astro import *

In [0]:
# obs = SubaruHSC.text_observation_reader().read(HSC_FILE)
# obs.data.shape


# This could be done nicer with a FeatherObservationReader or similar
obs = Observation()
obs.append_photometry(SubaruHSC.photometry())
# fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_obs.feather')
fn = os.path.join(OUTPUT_PATH, f'for_obs.feather')
obs._set_data(DataFrameSerializer().read(fn))
obs.data.shape

In [0]:
obs_mask = obs.data['priority'] > 0
obs_mask.shape, obs_mask.sum()

## PN

In [0]:
from pfs.ga.targeting.data import Observation

In [0]:
pn = Observation()
pn._set_data(pd.read_csv(PN_FILE, sep=r'\s+', comment='#', skiprows=8, names=['SlNo', 'RA', 'DEC', 'mag_5007', 'flux_5007', 'mag_g', 'flag']))
pn.data.rename(columns={'RA': 'RA', 'DEC': 'Dec'}, inplace=True)

pn.data['objid'] = pn.data['SlNo'] + 99000
pn.data['priority'] = 0
pn.data['exp_time'] = 9000

pn.data

## Sky

In [0]:
from pfs.ga.targeting.io import PfsSkyReader, PfsFluxStdReader

In [0]:
r = PfsSkyReader()
sky = r.read(SKY_FILE)
sky.data.shape

In [0]:
r = PfsFluxStdReader()
fluxstd = r.read(FLUXSTD_FILE)
fluxstd.data.shape

In [0]:
# from pfs.ga.targeting.selection import MagnitudeSelection, ColorSelection
# import copy

# fstars = MagnitudeSelection(cmd.axes[1], 18, 20).apply(obs) & ColorSelection(cmd.axes[0], 0.09, 0.46).apply(obs) & (obs.data['cli'] <= 0.5) & (obs.data['clg'] <= 0.5)
# fluxstd = copy.deepcopy(obs)
# fluxstd._set_data(obs.get_data(mask=fstars))
# fluxstd.data

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
# fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='b')
fov.plot_observation(ax, sky, c='lightgray')
fov.plot_observation(ax, fluxstd, c='r')

# fov.plot_observation(ax, pn, c='r', size=3, marker='x')

pfi = SubaruPFI(instrument_options={'layout': 'calibration'})
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

ax.set_aspect('equal', adjustable='datalim')

# f.tight_layout()

# Load the assignments

In [0]:
from pfs.ga.targeting.io import DataFrameSerializer

# fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_assignments.feather')
fn = os.path.join(OUTPUT_PATH, f'for_assignments.feather')

s = DataFrameSerializer()
assignments = s.read(fn)

assignments

In [0]:
# Count the number of assigned fibers for each visit
assignments.groupby('visit_idx').size()

In [0]:
# Plot focal plane and sky coordinates

f, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=240)

axs[0].plot(assignments['fp_x'], assignments['fp_y'], 'o', ms=1, markeredgewidth=0)
axs[0].plot(assignments['fp_x'][100], assignments['fp_y'][100], 'or', ms=1, markeredgewidth=0)
axs[0].set_aspect('equal', adjustable='datalim')
axs[0].set_title("Focal plane")

axs[1].plot(assignments['RA'], assignments['Dec'], 'o', ms=1, markeredgewidth=0)
axs[1].plot(assignments['RA'][100], assignments['Dec'][100], 'or', ms=1, markeredgewidth=0)
# axs[1].set_aspect('equal', adjustable='datalim')
axs[1].set_title("Sky")

# Generate pfsDesign files

In [0]:
# Total number of fiber allocations
# This should be the number of total visits times the number of fibers the reach the spectrographs 2458
assignments.shape

In [0]:
assignments['pointing_idx'].unique().size, assignments['visit_idx'].unique().size

In [0]:
for c in assignments.columns:
    print(c, assignments[c].dtype)

### Calculate the fluxes for the observations

The design files need the fluxes which are not processed though netflow, so we need to join them here.

Fluxes are represented as a list of numbers, one list for each target, along with a list of filter names.

Filter names are postfixed with the filter system, e.g. "r_ps1".

pfsDesign expects three difference fluxes:
* PSF flux
* Total flux
* Fiber flux

We are going to store the lists in the DataFrame we pass to `get_pfsDesign`

In [0]:
# The HSC dSph data files only contain PSF magnitudes, we need to calculate the fluxes
# This will create columns like `obs_flux_hsc_g`, 'err_flux_hsc_g', etc.
obs.calculate_flux(force=False)

In [0]:
obs.data.columns

In [0]:
from pfs.ga.targeting.util.astro import *
import astropy.units as u

In [0]:
pn.data['flux_5007'] = (np.array((pn.data['mag_5007'])) * u.ABmag).to_value(u.nJy)
pn.data['flux_g'] = (np.array((pn.data['mag_g'])) * u.ABmag).to_value(u.nJy)

In [0]:
pn.data

In [0]:
assignments['targetid'].unique()

In [0]:
# Generate a list of unique target IDs
# Note that `target_id` is not unique in `assignments` because there can be repeated visits.

unique_targetid = pd.DataFrame({ 'targetid': assignments['targetid'].unique() })
unique_targetid.set_index('targetid', inplace=True)
unique_targetid.shape

In [0]:
# Filter down observations to only those that are in the assignments
# Also exclude objects which were selected as flux standards for some reason
assignments_obs = obs.data.set_index('objid').join(unique_targetid, how='inner')

# Convert index back to `objid` column
assignments_obs.reset_index(inplace=True, names='objid')

# Set additional, missing columns
assignments_obs['epoch'] = 'J2000.0'                    # Why is it a string? Strings are used for the equinox not epoch
assignments_obs['tract'] = 0
assignments_obs['patch'] = '0,0'
assignments_obs['catid'] = 15001                        # Where do we get these from?
assignments_obs['proposalid'] = 'SSP_GA_dSph'

assignments_obs

In [0]:
# Convert flux columns into columns of lists

# The design file contains the flux for each target but the list of filters can vary from target to target

def flux_to_list(row, prefix='obs'):
    return [ row[f'{prefix}_hsc_{b}'] for b in [ 'g', 'i', 'nb515'] ]

def filter_to_list(row):
    return [ 'hsc_g', 'hsc_i', 'hsc_nb515' ]

for prefix in [ 'psf', 'fiber', 'total' ]:
    assignments_obs[f'{prefix}_flux'] = assignments_obs.apply(lambda row: flux_to_list(row, 'obs_flux'), axis=1)
    assignments_obs[f'{prefix}_flux_err'] = assignments_obs.apply(lambda row: flux_to_list(row, 'err_flux'), axis=1)

assignments_obs['filter'] = assignments_obs.apply(filter_to_list, axis=1)

In [0]:
assignments_obs.columns

## Convert columns of the PN

In [0]:
# Filter down observations to only those that are in the assignments
assignments_pn = pn.data.set_index('objid').join(unique_targetid, how='inner')

# Convert index back to `objid` column
assignments_pn.reset_index(inplace=True, names='objid')

# Set additional, missing columns
assignments_pn['epoch'] = 'J2000.0'                    # Why is it a string? Strings are used for the equinox not epoch
assignments_pn['tract'] = 0
assignments_pn['patch'] = '0,0'
assignments_pn['catid'] = 99001                        # Where do we get these from?
assignments_pn['proposalid'] = 'SSP_GA_PN'

assignments_pn

In [0]:
# Convert flux columns into columns of lists

# The design file contains the flux for each target but the list of filters can vary from target to target

def flux_to_list(row, prefix='flux'):
    return [ row[f'{prefix}_{b}'] for b in [ 'g', '5007'] ]

def filter_to_list(row):
    return [ 'g', '5007' ]

for prefix in [ 'psf', 'fiber', 'total' ]:
    assignments_pn[f'{prefix}_flux'] = assignments_pn.apply(lambda row: flux_to_list(row), axis=1)
    assignments_pn[f'{prefix}_flux_err'] = [[0.0, 0.0]] * assignments_pn.shape[0]

assignments_pn['filter'] = assignments_obs.apply(filter_to_list, axis=1)

In [0]:
assignments_pn.columns

In [0]:
assignments_pn[['filter', 'psf_flux', 'psf_flux_err', 'total_flux', 'total_flux_err']]

### Convert the columns of flux standards

In [0]:
fluxstd.data.shape

In [0]:
fluxstd.data.columns

In [0]:
# Join the flux standards with the assignments to work with a smaller dataset
assignments_fluxstd = fluxstd.data.set_index('objid').join(unique_targetid, how='inner')

# Convert index back to `objid` column
assignments_fluxstd.reset_index(inplace=True, names='objid')

# Set additional, missing columns'
if 'epoch' not in assignments_fluxstd:
    assignments_fluxstd['epoch'] = 'J2000.0'  # We get this for the flux standards
assignments_fluxstd['tract'] = 0
assignments_fluxstd['patch'] = '0,0'
assignments_fluxstd['catid'] = -1
assignments_fluxstd['proposalid'] = 'N/A'

assignments_fluxstd

In [0]:
# PS1 flux standards
assignments_fluxstd[[ f'filter_{f}' for f in 'grizyj' ]]

# HSC flux standards
# assignments_fluxstd[[ f'obs_hsc_{f}' for f in 'gi' ]]

In [0]:
# Convert flux columns into columns of lists

# Use with PS1 flux standards

def flux_to_list(row, prefix='psf_flux'):
    fluxes = []
    for f in 'grizyj':
        if row[f'filter_{f}'] is not None:
            fluxes.append(row[f'{prefix}_{f}'])
    return fluxes
        

def filter_to_list(row):
    filters = []
    for f in 'grizyj':
        if row[f'filter_{f}'] is not None:
            filters.append(row[f'filter_{f}'])
    return filters


for prefix in [ 'psf', 'fiber', 'total' ]:
    assignments_fluxstd[f'{prefix}_flux'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'psf_flux'), axis=1)
    assignments_fluxstd[f'{prefix}_flux_err'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'psf_flux_error'), axis=1)

assignments_fluxstd['filter'] = assignments_fluxstd.apply(filter_to_list, axis=1)

In [0]:
# Use with HSC flux standards

# def flux_to_list(row, prefix='obs_hsc_'):
#     fluxes = []
#     for f in 'gi':
#         fluxes.append(row[f'{prefix}_{f}'])
#     return fluxes

# def filter_to_list(row):
#     filters = [ 'hsc_g', 'hsc_i' ]
#     return filters

# for prefix in [ 'psf', 'fiber', 'total' ]:
#     assignments_fluxstd[f'{prefix}_flux'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'obs_hsc'), axis=1)
#     assignments_fluxstd[f'{prefix}_flux_err'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'err_hsc'), axis=1)

# assignments_fluxstd['filter'] = assignments_fluxstd.apply(filter_to_list, axis=1)

In [0]:
assignments_fluxstd.columns

In [0]:
assignments_fluxstd[['filter', 'psf_flux', 'psf_flux_err', 'total_flux', 'total_flux_err']]

### List of all targets

In [0]:
assignments_obs['objid']

In [0]:
assignments_fluxstd['objid']

In [0]:
# Pick only columns required for PfsDesign
columns = [ 'objid', 'epoch', 'proposalid', 'tract', 'patch', 'catid',
           'filter', 'psf_flux', 'psf_flux_err', 'fiber_flux', 'fiber_flux_err', 'total_flux', 'total_flux_err' ]

# Exclude any possible duplicates from assignments_obs that might be in assignments_fluxstd
# in case the flux standards were selected from the same original catalog
# Find all objid in assignments_obs that appear in assignments_fluxstd
assignments_obs_mask = ~assignments_obs['objid'].isin(assignments_fluxstd['objid'])
print('Duplicates', assignments_obs_mask.sum(), (~assignments_obs_mask).sum())


assignments_all = pd.concat([ assignments_obs[assignments_obs_mask][columns], assignments_fluxstd[columns] ])
# assignments_all = pd.concat([ assignments_obs[columns], assignments_pn[columns], assignments_fluxstd[columns] ])

In [0]:
for c in assignments_all.columns:
    print(c, assignments_all[c].dtype)

In [0]:
assignments_all

In [0]:
assignments_all[assignments_all['catid'] == 99001]

### Merge the assignments with the fluxes etc. calculated above

In [0]:
from pfs.ga.targeting.netflow.util import *
from pfs.datamodel import TargetType

In [0]:
# assignments.join(obs.data[cols], on=['targetid', 'objid'], how='left')

a = assignments.set_index('targetid')
b = assignments_all.set_index('objid')

# Only include columns that are not in a
cols = b.columns.difference(a.columns)
print(cols)

all_assignments = pd_to_nullable(a).join(pd_to_nullable(b[cols]), how='left')
all_assignments.reset_index(inplace=True, names='targetid')

for c in all_assignments.columns:
    print(c, all_assignments[c].dtype)

In [0]:
# Set the catid column to -1 for sky fibers and unassigned fibers
# This is necessary to be able to validate the design
all_assignments.loc[all_assignments['target_type'] == TargetType.SKY, 'catid'] = -1
all_assignments.loc[all_assignments['target_type'] == TargetType.UNASSIGNED, 'catid'] = -1
all_assignments.loc[all_assignments['target_type'] == TargetType.UNASSIGNED, 'prefix'] = 'ua'

In [0]:
all_assignments[all_assignments['prefix'] == 'sci'][['targetid', 'catid', 'target_type', 'fiber_status', 'filter']]

In [0]:
all_assignments[all_assignments['prefix'] == 'sky'][['targetid', 'catid', 'target_type', 'fiber_status', 'filter']]

In [0]:
all_assignments[all_assignments['prefix'] == 'cal'][['targetid', 'catid', 'target_type', 'fiber_status', 'filter']]

In [0]:
all_assignments['prefix'].unique()

In [0]:
all_assignments

In [0]:
for c in all_assignments.columns:
    print(c, all_assignments[c].dtype)

In [0]:
# Do some final polishing

# TODO: determine tract and patch from the coordinates
all_assignments['tract'] = 0
all_assignments['patch'] = '0,0'
all_assignments['obcode'] = '-1'

# Replace None with empty lists in columns containing lists
all_assignments['filter'] = all_assignments['filter'].apply(lambda x: x if isinstance(x, list) else [])
for prefix in ['fiber', 'psf', 'total']:
    all_assignments[f'{prefix}_flux'] = all_assignments[f'{prefix}_flux'].apply(lambda x: x if isinstance(x, list) else [])
    all_assignments[f'{prefix}_flux_err'] = all_assignments[f'{prefix}_flux_err'].apply(lambda x: x if isinstance(x, list) else [])

In [0]:
mask = (all_assignments['visit_idx'] == 0) & (all_assignments['targetid'] != -1)

# Count the number of target_id occurances in all_assignments
all_assignments['targetid'][mask].value_counts()

# all_assignments[mask]

In [0]:
# Find all duplicate targetids in all_assignments
duplicate = all_assignments.duplicated(subset=['visit_idx', 'targetid'])
all_assignments[duplicate & (all_assignments['targetid'] != -1)]

In [0]:
from pfs.ga.targeting.netflow import Design

designs = []
for visit in visits:
    d = Design.create_pfsDesign_visit(visit, all_assignments)
    d.designName = f'ga_{galaxy.ID}'
    designs.append(d)

    print(hex(d.pfsDesignId), d.designName, d.filename)
    print()

In [0]:
designs[0].__dict__

In [0]:
for d in designs:
    print(hex(d.pfsDesignId), d.fiberId.shape)

    # Find any duplicate objids in the design
    duplicate = pd.DataFrame({'objid': d.objId}).duplicated()
    print(d.objId[duplicate & (d.objId != -1)])

    

In [0]:
for d in designs:
    print(hex(d.pfsDesignId), d.fiberId.shape)

    # Find any duplicate objids in the design
    duplicate = pd.DataFrame({'fiberId': d.fiberId}).duplicated()
    print(d.fiberId[duplicate])

    

In [0]:
OUTPUT_PATH

In [0]:
# Save to FITS
for d in designs:
    d.write(dirName=OUTPUT_PATH)
    print(d.filename, d.raBoresight, d.decBoresight, d.posAng)

### Read back the file

In [0]:
from astropy.io import fits
from astropy.table import Table

# Load back the design file with astropy for direct inspection
fn
with fits.open(os.path.join(OUTPUT_PATH, designs[0].filename), memmap=False) as hdul:
    hdul.info()
    # print(hdul[0].header)
    # print(hdul[1].header)
    # print(hdul[2].header)

    data = Table(hdul[1].data)
    
data.info()

In [0]:
data

In [0]:
designs[0].pfsDesignId

In [0]:
from pfs.datamodel import PfsDesign

fn = os.path.join(OUTPUT_PATH, designs[0].filename)

PfsDesign.read(designs[0].pfsDesignId, dirName=OUTPUT_PATH)